In [3]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from registration_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk


In [4]:
def generate_annotation_file_from_aligned_atlas(stack_m, stack_f, classifier_setting_m, classifier_setting_f,
                                               warp_setting, level=.5, sample_every=1):
    """
    Compute the structure contours on a fixed stack according to the atlas aligned to it.
    The structure contours are stored as a pandas dataframe. 
    Rows are section indices and columns are structure names.
    Structure names are unsided only.
    Also save this dataframe as a hdf file. 
    The file location can be retrieved by DataManager.get_annotation_filepath().
    
    Returns:
        dataframe: the structure contours
    """
        
    warped_volumes = DataManager.load_transformed_volume_all_known_structures(stack_m=stack_m, stack_f=stack_f,
                                    classifier_setting_m=classifier_setting_m,
                                    classifier_setting_f=classifier_setting_f,
                                    warp_setting=warp_setting, sided=True)
    downsample_factor = 32
    
    xmin_vol_f, _, ymin_vol_f, _, zmin_vol_f, _ = \
    DataManager.load_original_volume_bbox(stack=stack_f, vol_type='score', structure='7N', downscale=downsample_factor, classifier_setting=classifier_setting_f)
    
    structure_contours = get_structure_contours_from_aligned_atlas(warped_volumes, downsample_factor=downsample_factor,
                                                         volume_origin=(xmin_vol_f, ymin_vol_f, zmin_vol_f),
                                                        sections=metadata_cache['valid_sections'][stack_f], 
                                                         level=level, sample_every=sample_every)

    del warped_volumes
    
    import uuid
    import datetime
    
    contours_df = {}
    for sec, row in structure_contours.iteritems():
        for name_s, vertices in row.iteritems():
            id_ = str(uuid.uuid4().fields[-1])
            side = get_side_from_label(name_s)
            if side is None:
                side = 'S'
            contour_entry = {
                'name': convert_to_original_name(name_s),
                'label_position': vertices.mean(axis=0),
                'side': side,
                'creator': 'auto',
                'time_created': datetime.datetime.now().strftime("%m%d%Y%H%M%S"),
                'edits': [],
                'vertices': vertices,
                'downsample': 1,
                'flags': 1, #1 means contour is unconfirmed.
                'section': sec,
                'filename': metadata_cache['sections_to_filenames'][stack_f][sec],
                'orientation': 'sagittal',
                'parent_structure': [],
                'side_manually_assigned': False,
                'id': id_
            }
            contours_df[id_] = contour_entry
        
    from pandas import DataFrame
    annotation_dataframe = DataFrame(contours_df).T
    
    annotation_df_fp = DataManager.get_annotation_filepath(stack=stack_f, by_human=False, stack_m=stack_m, 
                                                           classifier_setting_m=classifier_setting_m,
                                                          classifier_setting_f=classifier_setting_f,
                                                          warp_setting=warp_setting)
    save_hdf_v2(annotation_dataframe, annotation_df_fp)
    upload_to_s3(annotation_df_fp)

In [5]:
classifier_id = 37
warp_setting = 8

In [13]:
generate_annotation_file_from_aligned_atlas(stack_m='atlasV3', stack_f='MD635', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting,
                                           level=.5, sample_every=1)

rm -rf /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_sp5.bp && mkdir -p /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes


Child returned 0


aws s3 cp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_sp5.bp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_sp5.bp


Child returned 1
0.96 seconds.
File does not exist: /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_sp5.bp
[Errno 2] No such file or directory: '/shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_sp5.bp'
Score volume for sp5 does not exist.


rm -rf /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_outerContour.bp && mkdir -p /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes


...

3 contours ([7, 5, 25]) of DC_R is extracted at level=0.50 on section 298.
Some probability mass of Sp5I_R are on section 298 but no contour is extracted at level=0.50.
Some probability mass of IC are on section 298 but no contour is extracted at level=0.50.
Some probability mass of SC are on section 298 but no contour is extracted at level=0.50.
Computing structure contours for section 299...
2 contours ([229, 5]) of VLL_R is extracted at level=0.50 on section 299.
2 contours ([37, 5]) of DC_R is extracted at level=0.50 on section 299.
Some probability mass of Sp5I_R are on section 299 but no contour is extracted at level=0.50.
Some probability mass of IC are on section 299 but no contour is extracted at level=0.50.
Some probability mass of SC are on section 299 but no contour is extracted at level=0.50.
Computing structure contours for section 300...
2 contours ([205, 5]) of LRt_R is extracted at level=0.50 on section 300.
3 contours ([221, 5, 5]) of VLL_R is extracted at level=0.50 

Computing structure contours for section 317...
5 contours ([13, 39, 5, 9, 9]) of PBG_R is extracted at level=0.50 on section 317.
2 contours ([157, 5]) of Sp5C_R is extracted at level=0.50 on section 317.
4 contours ([287, 5, 5, 5]) of Sp5O_R is extracted at level=0.50 on section 317.
Computing structure contours for section 318...
Some probability mass of PBG_R are on section 318 but no contour is extracted at level=0.50.
3 contours ([5, 235, 5]) of Sp5O_R is extracted at level=0.50 on section 318.
2 contours ([117, 5]) of SNR_R is extracted at level=0.50 on section 318.
Computing structure contours for section 319...
2 contours ([221, 5]) of Sp5O_R is extracted at level=0.50 on section 319.
2 contours ([69, 5]) of SNR_R is extracted at level=0.50 on section 319.
2 contours ([235, 5]) of DC_R is extracted at level=0.50 on section 319.
Computing structure contours for section 320...
3 contours ([5, 7, 19]) of Sp5C_R is extracted at level=0.50 on section 320.
2 contours ([185, 5]) of S

Some probability mass of DC_R are on section 342 but no contour is extracted at level=0.50.
Computing structure contours for section 343...
2 contours ([209, 5]) of VCA_R is extracted at level=0.50 on section 343.
2 contours ([183, 5]) of VCP_R is extracted at level=0.50 on section 343.
Computing structure contours for section 344...
2 contours ([181, 5]) of VCP_R is extracted at level=0.50 on section 344.
Computing structure contours for section 345...
Computing structure contours for section 346...
Computing structure contours for section 347...
Computing structure contours for section 348...
3 contours ([261, 5, 5]) of VCA_R is extracted at level=0.50 on section 348.
2 contours ([159, 5]) of VCP_R is extracted at level=0.50 on section 348.
Computing structure contours for section 349...
Computing structure contours for section 350...
Computing structure contours for section 351...
3 contours ([263, 5, 5]) of VCA_R is extracted at level=0.50 on section 351.
Computing structure contou

aws s3 cp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_outerContour.bp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_outerContour.bp
aws s3 cp /shared/CSHL_labelings_v3/MD635/annotation_atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37.hdf s3://mousebrainatlas-data/CSHL_labelings_v3/MD635/annotation_atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37.hdf


Child returned 0
0.85 seconds.


In [26]:
# Load annotations created by human
contours_df, _ = DataManager.load_annotation_v3(stack='MD594', by_human=True)

'No object named structures in the file'


Annotation has no structures.


In [6]:
# Load annotations created by our system.
contours_df, _ = DataManager.load_annotation_v3(stack_m='atlasV3', by_human=False, 
                                                      stack='MD635', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting)

In [10]:
print sorted(contours_df[contours_df['name'] == 'SNR']['section'])

[120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320]


# Generate structure annotation to grid index lookup

In [15]:
from learning_utilities import generate_annotation_to_grid_indices_lookup
grid_index_lookup = generate_annotation_to_grid_indices_lookup(stack_m='atlasV3', by_human=False, stack='MD635', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting, 
                                                               surround_margins=[500])

rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_masks && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp --recursive s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_masks /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_masks


Child returned 0
Child returned 0
2.79 seconds.
Computing grid indices lookup for section 77...
Computing grid indices lookup for section 78...
Computing grid indices lookup for section 79...
Computing grid indices lookup for section 80...
Computing grid indices lookup for section 81...
Computing grid indices lookup for section 82...
Computing grid indices lookup for section 83...
Computing grid indices lookup for section 84...
Computing grid indices lookup for section 85...
Computing grid indices lookup for section 86...
Computing grid indices lookup for section 87...
Computing grid indices lookup for section 88...
Computing grid indices lookup for section 89...
Computing grid indices lookup for section 90...
Computing grid indices lookup for section 91...
Computing grid indices lookup for section 92...
Computing grid indices lookup for section 94...
Computing grid indices lookup for section 95...
Computing grid indices lookup for section 96...
Computing grid indices lookup for sectio

Computing grid indices lookup for section 251...
Computing grid indices lookup for section 252...
Computing grid indices lookup for section 253...
Computing grid indices lookup for section 254...
Computing grid indices lookup for section 255...
Computing grid indices lookup for section 256...
Computing grid indices lookup for section 257...
Computing grid indices lookup for section 258...
Computing grid indices lookup for section 259...
Computing grid indices lookup for section 260...
Computing grid indices lookup for section 261...
Computing grid indices lookup for section 262...
Computing grid indices lookup for section 263...
Computing grid indices lookup for section 264...
Computing grid indices lookup for section 265...
Computing grid indices lookup for section 266...
Computing grid indices lookup for section 267...
Computing grid indices lookup for section 268...
Computing grid indices lookup for section 269...
Computing grid indices lookup for section 270...
Computing grid indic

In [21]:
grid_indices_lookup_fp = DataManager.get_annotation_to_grid_indices_lookup_filepath(stack_m='atlasV3', 
                                                                                    by_human=False, stack='MD635', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting)
save_hdf_v2(grid_index_lookup, grid_indices_lookup_fp)
upload_to_s3(grid_indices_lookup_fp)

aws s3 cp /shared/CSHL_labelings_v3/MD635/annotation_atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_grid_indices_lookup.hdf s3://mousebrainatlas-data/CSHL_labelings_v3/MD635/annotation_atlasV3_down32_scoreVolume_clf_37_warp_8_MD635_down32_scoreVolume_clf_37_grid_indices_lookup.hdf


Child returned 0
3.84 seconds.
